In [1]:
import os
from PIL import Image
import torch
import torchvision.transforms.functional as tf
from tqdm import tqdm

In [2]:
base = "data/"
input_folders = ["rainfall/", "mean_temp/", "soil_moisture/", "runoff/", "vp/", "wind_speed/"]
output_folder = "FD/"

Input Processing

In [3]:
input_files = []
for input_folder in input_folders:
    input_files.append(os.listdir(base + input_folder))

num_files = len(input_files[0])

In [24]:
feature_min = []
feature_max = []
feature_std = []

for i in range(10):
    for j in range(len(input_folders)):
        file_path = base + input_folders[j] + input_files[j][i]

        im = Image.open(file_path)
        arr = tf.to_tensor(im)

        if len(feature_min) < len(input_folders):
            feature_min.append(arr[arr > -1e30].min())
            feature_max.append(arr[arr > -1e30].max())
            feature_std.append(arr[arr > -1e30].std())
        else:
            feature_min[j] = min(feature_min[j], arr[arr > -1e30].min())
            feature_max[j] = max(feature_max[j], arr[arr > -1e30].max())
            feature_std[j] = ((feature_std[j] * i) + arr[arr > -1e30].std()) / (i + 1)

In [25]:
print(feature_min)
print(feature_max)
print(feature_std)

[tensor(0.), tensor(6.0500), tensor(0.1572), tensor(2.8638e-14), tensor(3.3230), tensor(0.4958)]
[tensor(235.), tensor(36.8000), tensor(241.7987), tensor(112.6599), tensor(36.6870), tensor(5.0300)]
[tensor(6.1211), tensor(4.1530), tensor(61.3320), tensor(2.2251), tensor(5.3199), tensor(0.5112)]


In [26]:
input_files = []
for input_folder in input_folders:
    input_files.append(os.listdir(base + input_folder))

num_files = len(input_files[0])

for i in tqdm(range(num_files)):
    input_feature = []
    for j in range(len(input_folders)):
        file_path = base + input_folders[j] + input_files[j][i]

        im = Image.open(file_path)
        arr = tf.to_tensor(im)
        arr[arr < -1e30] = 0
        arr = arr / feature_max[j]
        input_feature.append(arr)
    input_feature = torch.cat(input_feature)
    torch.save(input_feature, base + "input/" + input_files[0][i][:-7] + "pt")

100%|██████████| 8035/8035 [1:05:01<00:00,  2.06it/s]


In [32]:
input_files = os.listdir(base + "input/")
inp = torch.load(base + "input/" + input_files[0])
print(inp.shape)
print(inp.min())
print(inp.max())

torch.Size([6, 676, 407])
tensor(0.)
tensor(0.9988)


Output Processing

In [12]:
output_files = os.listdir(base + output_folder)

num_files = len(output_files)

In [13]:
for i in tqdm(range(num_files)):
    file_path = base + output_folder + output_files[i]
    im = Image.open(file_path)
    arr = tf.to_tensor(im)
    arr[arr <= 1] = False
    arr[arr == 2] = True
    torch.save(arr, base + "output/" + output_files[i][:-11] + "pt")

100%|██████████| 7967/7967 [12:14<00:00, 10.85it/s]


In [14]:
output_files = os.listdir(base + "output/")
out = torch.load(base + "output/" + output_files[0])
print(out.shape, out.dtype)
print(torch.unique(out, return_counts=True))

torch.Size([1, 676, 407]) torch.float32
(tensor([0., 1.]), tensor([271682,   3450]))


In [16]:
input_folder = base + "input/"
output_folder = base + "output/"
input_files = os.listdir(input_folder)
output_files = os.listdir(output_folder)
for i in range(len(output_files)):
    assert output_files[i] == input_files[i+68]